In [3]:
# Loading Pre-requisites

import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout,SeparableConv2D,BatchNormalization, Activation, Dense
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import os
import glob

In [4]:
# Specifying No of Classes in dataset and a few Hyper-parameters

#no_of_categories = 100

num_class = 100

#chosen_batch_size = 64
#chosen_learning_rate = 1e-3

In [5]:
# Loading Training Dataset - 100 species of plants- 450 images of each species in 100 folders named by Plant Species -
#using UHMANOA100 Dataset
# Also generating Labels i.e. Plant Species for each of these 45*100 = 45000 images 

def get_folders_names(folder):
    folders = []
    for filename in os.listdir(folder):
        folders.append(filename)
        ## print(filename)
        #img = cv2.imread(os.path.join(folder, filename))
        # print(img)
        #if img is not None:
        #    images.append(img)
    # print(shape(images[0]))
    return folders

fold = get_folders_names('train')

# Loading Training Set and Generating labels for training data - same as folder name

data = []
labels = []

for folder in fold:
    images = glob.glob('./train/%s/*.*'%folder)
    for i in range(len(images)):
        if (i+1)%3 == 0:
        #if i%2 == 0:
            image = cv2.imread(images[i])
            try :
                image = cv2.resize(image, (224,224))
                image = tf.keras.preprocessing.image.img_to_array(image)
                data.append(image)
                labels.append(folder)
            except :
                pass
    #print(len(images),folder)


data = np.array(data)
label = np.array(labels)

print("Training Input Data Array size")
print(data.shape)

dict1 = {}
label_dict = {}
for i in range(len(fold)):
    folder1 = str(fold[i])
    dict1[folder1] = i
    label_dict[i] = folder1


label = np.vectorize(dict1.get)(label)
print("Training Labels : size of Array")
print(label.shape)

train_images_reshaped = data.reshape((-1, 224, 224,3))
train_labels_reshaped = to_categorical(label, num_classes=num_class)

Training Input Data Array size
(15000, 224, 224, 3)
Training Labels : size of Array
(15000,)


In [6]:
# Creating Image Classification Model

# Using MobileNet as base model, we add a few additional layers at the end. The Aim is to use Transfer Learning using MobileNet
# trained on ImageNet Dataset.
# We freeze the weights of MobileNet layers and train the final few additional layers.

#Base model without Fully connected Layers
base_model = MobileNet(include_top=False, weights='imagenet', input_shape=(224,224,3))
x=base_model.output
# Add some new Fully connected layers to
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
x = Dropout(0.25)(x)
x=Dense(512,activation='relu')(x)
x = Dropout(0.25)(x)
preds=Dense(num_class, activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

In [7]:
# Freezing the MobileNet layers and Training the final few additional layers using out dataset

for layer in model.layers[:78]:
    layer.trainable=False
for layer in model.layers[78:]:
    layer.trainable=True

In [8]:
# Compiling the model 

#Training for 50 epochs
epochs = 30

learning_rate = 0.0001

decay_rate = learning_rate / epochs

opt = Adam(learning_rate = learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay_rate, amsgrad=False)

model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=[tf.keras.metrics.categorical_accuracy])

In [9]:
print("Start of Training")

%matplotlib inline
import matplotlib.pyplot as plt


chosen_batch_size = 64

model.fit(train_images_reshaped, train_labels_reshaped,batch_size= chosen_batch_size,epochs=epochs)



# summarize history for accuracy
#plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
#plt.title('model accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'val'], loc='upper left')
#plt.show()
# summarize history for loss
#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'val'], loc='upper left')
#plt.show()

Start of Training
Epoch 1/30
235/235 [==============================] - 58s 193ms/step - loss: 4.5502 - categorical_accuracy: 0.0363
Epoch 2/30
235/235 [==============================] - 36s 155ms/step - loss: 3.7202 - categorical_accuracy: 0.1553
Epoch 3/30
235/235 [==============================] - 36s 155ms/step - loss: 3.1307 - categorical_accuracy: 0.2618
Epoch 4/30
235/235 [==============================] - 36s 155ms/step - loss: 2.6982 - categorical_accuracy: 0.3453
Epoch 5/30
235/235 [==============================] - 37s 155ms/step - loss: 2.3024 - categorical_accuracy: 0.4341
Epoch 6/30
235/235 [==============================] - 36s 155ms/step - loss: 1.9600 - categorical_accuracy: 0.5145
Epoch 7/30
235/235 [==============================] - 36s 155ms/step - loss: 1.6783 - categorical_accuracy: 0.5844
Epoch 8/30
235/235 [==============================] - 36s 154ms/step - loss: 1.4302 - categorical_accuracy: 0.6445
Epoch 9/30
235/235 [==============================] - 36s 155m

In [10]:
#Loading test data

def load_images_from_folder(folder):
    images = []
    lbs = []
    i = 0
    for filename in os.listdir(folder):
        if i%3 == 0:
            img = cv2.imread(os.path.join(folder,filename))
            if img is not None:
                img = cv2.resize(img, (224,224))
                img = tf.keras.preprocessing.image.img_to_array(img)
                images.append(img)
                lab = filename.split('(')[0]
                lbs.append(lab)
        i += 1

    return images, lbs

test_data, test_labels = load_images_from_folder('test')



test_data = np.array(test_data)
test_label = np.array(test_labels)
print(test_data.shape)



dict2 = {}
label_dict2 = {}
for i in range(len(fold)):
    folder1 = str(fold[i])
    dict2[folder1] = i
    label_dict2[i] = folder1

test_label = np.vectorize(dict2.get)(test_label)
print(test_label.shape)

test_images_reshaped = test_data.reshape((-1, 224, 224,3))
test_labels_reshaped = to_categorical(test_label, num_classes= num_class)

(927, 224, 224, 3)
(927,)


In [11]:
# test accuracy

(test_loss, test_accuracy) = model.evaluate(test_images_reshaped,test_labels_reshaped,
                                            batch_size=chosen_batch_size, verbose=1)
print("[INFO] Test Accuracy: {:.2f}%".format(test_accuracy * 100))
print("[INFO] Loss: {}".format(test_loss))

15/15 [==============================] - 10s 667ms/step - loss: 3.7110 - categorical_accuracy: 0.3085
[INFO] Test Accuracy: 30.85%
[INFO] Loss: 3.710979461669922
